# Encoder multi-head attention

Ya hemos vísto cómo funciona el `Scaled Dot-Product Attention`, por lo que ya podemos abordar el `Multi-Head Attention` del encoder

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_encoder_multi_head_attention.png" alt="Multi-Head Attention" style="width:425px;height:626px;">
  <img src="Imagenes/multi-head_attention.png" alt="Multi-Head Attention" style="width:501px;height:623px;">
</div>

Viendo la arquitectura, podemos ver que `K`, `Q` y `V` pasan por un bloque `Linear` y además vemos que tanto los bloques `Linear` y el bloque `Scaled Dot-Product Attention` se repite varias veces, vamos a ver por qué se hace esto

En primer lugar hay que explicar qué es el bloque `Linear` que se ve en la arquitectura, pues es simplemente una capa fully connected de las que explicamos al principio del todo, y ahora veremos por qué

## Proyecciones de las entradas

<div style="text-align:center;">
  <img src="Imagenes/multi-head_attention_proyection.png" alt="Proyection Multi-Head Attention" style="width:501px;height:623px;">
</div>

Como hemos explicado, en la capa de `Input Embedding` se convierte el token a un vector de `n` dimensiones. De manera que por ejemplo puede que una dimensión determine si una palabra es un objeto inanimado o un ser vivo, otra podría capturar si es un objeto tangible o un concepto abstracto, otra podría capturar si la palabra tiene connotaciones positivas o negativas, mientras que otra podría capturar si una palabra es un sustantivo, verbo, adjetivo, etc., otra dimensión podría capturar si la palabra está en singular o plural y otra dimensión podría capturar el tiempo verbal (presente, pasado, futuro). Las tres primeras dimensiones capturan información de la semántica de la palabra, mientras que las tres últimas capturan información de la sintaxis

Por lo que a la hora de introducir las matrices `K`, `Q` y `V` al `Scaled Dot-Product Attention` a lo mejor es mejor que entren juntas las tres primeras dimensiones y por otro lado las tres últimas

Sin embargo, no sabemos exactamente cómo el input embedding construye las dimensiones. En nuestro caso de momento estamos cogiendo el `input embedding` ya entrenado de BERT, por lo que podríamos hacer un estudio, pero son 768 dimensiones, así que sería mucho trabajo hacer el estudio de cada dimensión y luego buscar relaciones entre las dimensiones. Pero es que además, en la realidad, el `input embedding` no está entrenado, va cambiando sus pesos durante el entrenamiento, por lo que ni podríamos hacer ese estudio.

Por lo que aquí entran las capas `Linear` que se ven en la arquitectura, nosotros decidimos en cuantos grupos vamos a dividir el embedding, esto lo determina la `h` que se ve a la derecha del `Scaled Dot-Product Attention`. Y esas capas `Linear`, simplemente serán matrices que se quedarán con unas dimensiones u otras, incluso con parte de unas o de otras, puede ser una mezcla. Pero lo importante, es que nosotros no decidimos qué dimensiones van con otras, sino que se hace automáticamente durante el entrenamiento del Transformer, de manera que se obtenga el mejor resultado posible

## Scaled Dot-Product Attention

Una vez se han dividido los embeddings en `h` grupos, se pasa por el `Scaled Dot-Product Attention` cada uno de los grupos, y se obtiene un resultado para cada uno de los grupos. Por lo que se obtienen `h` resultados

## Concatenación

<div style="text-align:center;">
  <img src="Imagenes/multi-head_attention_concat.png" alt="Concat Multi-Head Attention" style="width:501px;height:623px;">
</div>

Si en las proyecciones se divide el embedding en pequeñas dimensiones, ahora habrá que juntar toda esa información nuevamente, por eso se hace una concatenación de todas las matrices resultantes del `Scaled Dot-Product Attencion` en la capa de `Concat`

## Linear final

<div style="text-align:center;">
  <img src="Imagenes/multi-head_attention_linear.png" alt="Linear Multi-Head Attention" style="width:501px;height:623px;">
</div>

Por último se pasa todo por un bloque `Linear`, es decir, por una red fully connected.

Esto es porque los resultados de los `h` grupos se han concatenado uno detrás de otro, pero esa organización de los embeddings resultantes no tiene por qué ser la menjor, por lo que nuevamente se vuelve a pasar por una red fully connected para que se organice de la mejor manera posible.

## Implementación

Vamos a implementar la clase `Multi-Head Attention` con Pytorch

Primero volvemos a poner el código de la clase `Scaled Dot-Product Attention` que hemos hecho en el notebook anterior

In [1]:
import torch
import torch.nn as nn

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, key, query, value):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / torch.sqrt(torch.tensor(self.dim_embedding))
        # softmax
        attention_matrix = torch.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

Y ahora creamos la clase del `Multi-Head Attention`

In [16]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads

        # Aquí lo ideal sería crear tantas capas lineales como cabezas h haya, es decir, algo así:
        #   self.proyection_Q = [nn.Linear(dim_embedding, self.dim_proyection) for _ in range(heads)]
        #   self.proyection_K = [nn.Linear(dim_embedding, self.dim_proyection) for _ in range(heads)]
        #   self.proyection_V = [nn.Linear(dim_embedding, self.dim_proyection) for _ in range(heads)]
        # Aquí hemos creado h capas lineales a las que le entra la matriz de embedding (dim_embedding) y sale una matriz de dimensión dim_proyection
        # Sin embargo computacionalmente es lo mismo que hacer una única capa linear que le entre la matriz de embedding y salga una matriz de dimensión dim_embedding
        # porque internamente, la capa lineal va a hacer las combinaciones de dimensiones necesarias para que se junten las dimensiones del embedding que hagan
        # el entrenamiento más óptimo
        self.proyection_Q = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_K = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_V = nn.Linear(dim_embedding, dim_embedding)

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
        self.attention = nn.Linear(dim_embedding, dim_embedding)
    
    def forward(self, Q, K, V):
        """
        Args:
            Q: query vector
            K: key vector
            V: value vector

        Returns:
            output vector from multi-head attention
        """
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        proyection_Q = self.proyection_Q(Q).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, -1, self.heads, self.dim_proyection)
        
        # transpose to get dimensions bs * h * sl * d_model
        proyection_Q = proyection_Q.transpose(1,2)
        proyection_K = proyection_K.transpose(1,2)
        proyection_V = proyection_V.transpose(1,2)

        # calculate attention
        scaled_dot_product_attention = self.scaled_dot_product_attention(proyection_Q, proyection_K, proyection_V)
        
        # concatenate heads and put through final linear layer
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)

        # Final linear        
        output = self.attention(concat)
    
        return output

Vamos a obtener el embedding de una frase

In [18]:
import torch
from transformers import BertModel, BertTokenizer

def extract_embeddings(input_sentences, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    # tokenización de lote
    inputs = tokenizer(input_sentences, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    token_embeddings = outputs[0]
    
    # Los embeddings posicionales están en la segunda capa de los embeddings de la arquitectura BERT
    positional_encodings = model.embeddings.position_embeddings.weight[:token_embeddings.shape[1], :].detach().unsqueeze(0).repeat(token_embeddings.shape[0], 1, 1)

    embeddings_with_positional_encoding = token_embeddings + positional_encodings

    # convierte las IDs de los tokens a tokens
    tokens = [tokenizer.convert_ids_to_tokens(input_id) for input_id in inputs['input_ids']]

    return tokens, inputs['input_ids'], token_embeddings, positional_encodings, embeddings_with_positional_encoding

In [19]:
sentence1 = "I gave the dog a bone because it was hungry"
tokens1, input_ids1, token_embeddings1, positional_encodings1, embeddings_with_positional_encoding1 = extract_embeddings(sentence1)

Instanciamos un objeto de la clase `Multi-Head Attention`

In [20]:
dim_embedding = embeddings_with_positional_encoding1.shape[-1]
heads = 8
multi_head_attention = MultiHeadAttention(heads=heads, dim_embedding=dim_embedding)

In [21]:
attention = multi_head_attention(embeddings_with_positional_encoding1, embeddings_with_positional_encoding1, embeddings_with_positional_encoding1)
attention.shape

torch.Size([1, 12, 768])